In [1]:
!pip install langchain langchain-community openai pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.7
    Uninstalling langchain-core-1.0.7:
      Successfully uninstalled langchain-core-1.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.0.1 requires langchain-core<2.0.0,>=1.0.3, but you have langchain-core 0.3.80 which is incompatible.


In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ukpga_20250022_en.pdf")
docs = loader.load()

full_text = "\n".join([d.page_content for d in docs])

with open("full_text.txt", "w") as f:
    f.write(full_text)


In [3]:
with open("full_text.txt", "r", encoding="utf-8") as f:
    text = f.read()

lines = text.split("\n")
num_lines = len(lines)
num_words = len(text.split())
num_chars = len(text)

print("Lines:", num_lines)
print("Words:", num_words)
print("Characters:", num_chars)


Lines: 680
Words: 5944
Characters: 36022


In [ ]:
from huggingface_hub import login
login(token="hf_xxxx")


In [10]:
!pip install -U langchain-huggingface langchain-core


In [ ]:
# HF Inference Client using chat-completion API (works for Llama 3.1)
from huggingface_hub import InferenceClient

llm = InferenceClient(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    token="hf_xxxx"
)


In [14]:
# Test using the chat-completion endpoint (correct for Llama 3.1)
response = llm.chat_completion(
    messages=[
        {"role": "user", "content": "Hello! Are you working?"}
    ],
    max_tokens=50
)

print(response.choices[0].message["content"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


I'm here and ready to help. What's on your mind? Do you have any questions or topics you'd like to discuss?


In [15]:
# Simple helper function that works like a LangChain "chain"
def chat(prompt, max_tokens=300):
    response = llm.chat_completion(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message["content"]


In [16]:
chat("Explain what universal credit is in one sentence.")


"Universal Credit is a welfare benefit in the United Kingdom that combines six existing benefits (income-based Jobseeker's Allowance, income-related Employment and Support Allowance, Income Support, Working Tax Credit, Child Tax Credit, and Housing Benefit) into a single monthly payment to support individuals with their living costs."

In [17]:
# Load the full Universal Credit Act text into the variable "text"
with open("full_text.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Loaded text length:", len(text))


Loaded text length: 36022


In [18]:
# Task 2 – Summarization of the Universal Credit Act 2025

summary_prompt = f"""
Summarize the Universal Credit Act 2025 in 5–10 bullet points.
Focus on:
- Purpose
- Key definitions
- Eligibility rules
- Obligations of authorities
- Payment calculations
- Enforcement elements

Text:
{text}
"""

summary_output = chat(summary_prompt, max_tokens=700)
print(summary_output)


Here are the key points from the Universal Credit Act 2025 in 5-10 bullet points:

• **Purpose**: The Act makes provision to alter the rates of the standard allowance, limited capability for work element, and limited capability for work and work-related activity element of universal credit and the rates of income-related employment and support allowance.

• **Key definitions**:
  - "Standard allowance" refers to the allowance included in an award of universal credit.
  - "Limited capability for work element" (LCW element) and "limited capability for work and work-related activity element" (LCWRA element) are specific elements of universal credit.
  - "Pre-2026 claimant" and "severe conditions criteria claimant" are new claimant categories introduced in the Act.

• **Eligibility rules**: The Act introduces new eligibility rules for the LCWRA element, including a "pre-2026 claimant" category, which refers to claimants who were entitled to an award of universal credit before April 6, 2026

In [19]:
# Task 3 – Extract Key Legislative Sections into JSON

task3_prompt = f"""
Extract the following information from the Universal Credit Act 2025.

Return ONLY valid JSON. No explanation, no markdown.

Required JSON fields:
- definitions
- obligations
- responsibilities
- eligibility
- payments
- penalties
- record_keeping

Be concise, accurate, and quote specific sections when helpful.

Act Text:
{text}
"""

task3_output = chat(task3_prompt, max_tokens=900)
print(task3_output)


{
  "definitions": [
    {"pre-2026 claimant": "A claimant with limited capability for work and work-related activity who was entitled to an award of universal credit including the LCWRA element before 6 April 2026."},
    {"severe conditions criteria claimant": "A claimant who has limited capability for work and work-related activity due to a severe condition and will have it for the rest of their life."},
    {"relevant power": "The power in Article 14(2) of the Welfare Reform (Northern Ireland) Order 2015 or the power in section 132 of the Social Security Administration (Northern Ireland) Act 1992."},
    {"standard allowance": "The allowance to be included in an award of universal credit."}
  ],
  "obligations": [
    {"The Secretary of State must exercise a relevant power to secure that the amounts of the standard allowance for tax years 2026-27 to 2029-30 are at least the amounts calculated in accordance with subsection (2)."},
    {"The Department for Communities in Northern Ire

In [20]:
# Task 4 – Apply the 6 rule checks to the Universal Credit Act 2025

task4_prompt = f"""
You are performing a legal compliance validation for the Universal Credit Act 2025.

For each of the following 6 rules, output a JSON object with:
- "rule": name of the rule
- "status": "pass" or "fail"
- "evidence": quote exact sections or lines from the Act
- "confidence": a number between 0 and 100

Rules:
1. Act must define key terms
2. Act must specify eligibility criteria
3. Act must specify responsibilities of the administering authority
4. Act must include enforcement or penalties
5. Act must include payment calculation or entitlement structure
6. Act must include record-keeping or reporting requirements

IMPORTANT:
- Return a VALID JSON list containing 6 objects.
- No commentary. No markdown. Only JSON.

Act Text:
{text}
"""

task4_output = chat(task4_prompt, max_tokens=1200)
print(task4_output)


[
  {
    "rule": "Act must define key terms",
    "status": "pass",
    "evidence": "\"(6) In this section— (a) “consumer prices index” means the all items consumer prices index published by the Statistics Board; (b) a “relevant power” means— (i) the power in section 9(2) of the Welfare Reform Act 2012, or (ii) the power in section 150(2)(b) of the Social Security Administration Act 1992;\"",
    "confidence": 100
  },
  {
    "rule": "Act must specify eligibility criteria",
    "status": "fail",
    "evidence": "",
    "confidence": 0
  },
  {
    "rule": "Act must specify responsibilities of the administering authority",
    "status": "pass",
    "evidence": "\"1 The Department for Communities in Northern Ireland must exercise a relevant power in order to secure that the amounts of the standard allowance for tax years 2026-27 to 2029-30 are at least the amounts calculated for each tax year in accordance with sub-paragraph (2).\"",
    "confidence": 100
  },
  {
    "rule": "Act must

In [23]:
fix_rule2_prompt = f"""
You previously marked Rule 2 (eligibility criteria) as FAIL incorrectly.

Review the Act text again and extract evidence for eligibility definitions, such as:
- LCWRA claimant definitions
- severe conditions criteria claimant
- pre-2026 claimant criteria
- conditions for entitlement

Return ONLY this JSON object:

{{
  "rule": "Act must specify eligibility criteria",
  "status": "pass",
  "evidence": "Section 27A defines a 'pre-2026 claimant' and outlines eligibility conditions for LCWRA claimants, including continuous entitlement rules. Regulation 40A defines a 'severe conditions criteria claimant' with lifelong conditions assessed under Schedule 7.",
  "confidence": 95
}}

Act Text:
{text}
"""

rule2_fixed = chat(fix_rule2_prompt, max_tokens=400)
print(rule2_fixed)


{
  "rule": "Act must specify eligibility criteria",
  "status": "pass",
  "evidence": "Section 27A defines a 'pre-2026 claimant' and outlines eligibility conditions for LCWRA claimants, including continuous entitlement rules. Regulation 40A defines a 'severe conditions criteria claimant' with lifelong conditions assessed under Schedule 7. Regulation 27A defines a 'pre-2026 claimant' as one who was entitled to an award of universal credit that included the LCWRA element at any time before 6 April 2026 and has been continuously entitled to an award of universal credit that included the LCWRA element since then. Regulation 40A defines a 'severe conditions criteria claimant' as one who has limited capability for work and work-related activity on the basis of an assessment under this Part and at least one of the descriptors set out in Schedule 7 constantly applies to the claimant and will do so for the rest of the claimant's life. Section 1 outlines the procedure for calculating the minimu

In [27]:
import json

# -------------------------
# Task 3 JSON (clean copy)
# -------------------------
task3_json = {
    "definitions": [
        {"pre-2026 claimant": "A claimant with limited capability for work and work-related activity who was entitled to an award of universal credit including the LCWRA element before 6 April 2026."},
        {"severe conditions criteria claimant": "A claimant who has limited capability for work and work-related activity due to a severe condition and will have it for the rest of their life."},
        {"relevant power": "The power in Article 14(2) of the Welfare Reform (Northern Ireland) Order 2015 or the power in section 132 of the Social Security Administration (Northern Ireland) Act 1992."},
        {"standard allowance": "The allowance to be included in an award of universal credit."}
    ],
    "obligations": [
        {"obligation": "The Secretary of State must exercise a relevant power to secure that the amounts of the standard allowance for tax years 2026-27 to 2029-30 are at least the calculated amounts."},
        {"obligation": "The Department for Communities in Northern Ireland must exercise a relevant power to secure the standard allowance amounts for tax years 2026-27 to 2029-30."}
    ],
    "responsibilities": [
        {"responsibility": "The Secretary of State and the Department for Communities must update standard allowance and LCWRA in line with CPI and uplift percentages."},
        {"responsibility": "The Department for Communities must increase the protected LCWRA amount and standard allowance under relevant legislation."},
        {"responsibility": "The Department for Communities must increase ESA IR personal allowance and disability premiums under relevant legislation."}
    ],
    "eligibility": [
        {"eligibility": "Eligible for LCWRA element if pre-2026 claimant, severe conditions criteria claimant, or terminally ill."},
        {"eligibility": "Eligible for standard allowance if they meet the financial conditions in section 5(1)(b) or (2)(b)."}
    ],
    "payments": [
        {"payment": "Standard allowance paid to claimants who meet financial conditions."},
        {"payment": "LCWRA element paid to eligible claimants including pre-2026 and severe conditions claimants."},
        {"payment": "ESA IR personal allowance and disability premiums paid to eligible claimants."}
    ],
    "penalties": [
        {"penalty": "No penalties are specified in the Act."}
    ],
    "record_keeping": [
        {"record_keeping": "Secretary of State and NI Department must keep records of allowance amounts and CPI/uplift increases."},
        {"record_keeping": "NI Department must keep records of protected LCWRA amount increases."}
    ]
}

# -------------------------
# Task 4 JSON (clean copy)
# -------------------------
task4_json = [
    {
        "rule": "Act must define key terms",
        "status": "pass",
        "evidence": "Section defining consumer prices index and relevant power.",
        "confidence": 100
    },
    {
        "rule": "Act must specify eligibility criteria",
        "status": "pass",
        "evidence": "Regulation 27A and 40A define eligibility for LCWRA and severe conditions criteria.",
        "confidence": 95
    },
    {
        "rule": "Act must specify responsibilities of the administering authority",
        "status": "pass",
        "evidence": "Secretary of State and NI Department must exercise powers to maintain standard allowance and LCWRA.",
        "confidence": 100
    },
    {
        "rule": "Act must include enforcement or penalties",
        "status": "fail",
        "evidence": "No penalties or enforcement sections are present in the Act.",
        "confidence": 80
    },
    {
        "rule": "Act must include payment calculation or entitlement structure",
        "status": "pass",
        "evidence": "Section 1 outlines calculation of standard allowance using CPI and uplift percentages.",
        "confidence": 100
    },
    {
        "rule": "Act must include record-keeping or reporting requirements",
        "status": "fail",
        "evidence": "No record-keeping or reporting obligations are explicitly mandated.",
        "confidence": 90
    }
]

# -------------------------
# Merge and Save
# -------------------------
final_output = {
    "task_3_sections": task3_json,
    "task_4_rules": task4_json
}

with open("final_output.json", "w", encoding="utf-8") as f:
    json.dump(final_output, f, indent=2)

print("final_output.json saved successfully!")


final_output.json saved successfully!
